In [107]:
import pandas as pd
from flask import Flask
import numpy as np

In [2]:
#importing dataset from https://www.kaggle.com/datasets/nehaprabhavalkar/indian-food-101
data = pd.read_csv("indian_food.csv")
data.head()

,name,ingredients,diet,prep_time,cook_time,flavor_profile,course,state,region
0,Balu shahi,"Maida flour, yogurt, oil, sugar",vegetarian,45,25,sweet,dessert,West Bengal,East
1,Boondi,"Gram flour, ghee, sugar",vegetarian,80,30,sweet,dessert,Rajasthan,West
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",vegetarian,15,60,sweet,dessert,Punjab,North
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",vegetarian,15,30,sweet,dessert,Rajasthan,West
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",vegetarian,15,40,sweet,dessert,West Bengal,East


In [3]:
#dropping unwanted columns
data = data.drop(columns=['flavor_profile','region','state', 'diet'])
data

,name,ingredients,prep_time,cook_time,course
0,Balu shahi,"Maida flour, yogurt, oil, sugar",45,25,dessert
1,Boondi,"Gram flour, ghee, sugar",80,30,dessert
2,Gajar ka halwa,"Carrots, milk, sugar, ghee, cashews, raisins",15,60,dessert
3,Ghevar,"Flour, ghee, kewra, milk, clarified butter, su...",15,30,dessert
4,Gulab jamun,"Milk powder, plain flour, baking powder, ghee,...",15,40,dessert
...,...,...,...,...,...
250,Til Pitha,"Glutinous rice, black sesame seeds, gur",5,30,dessert
251,Bebinca,"Coconut milk, egg yolks, clarified butter, all...",20,60,dessert
252,Shufta,"Cottage cheese, dry dates, dried rose petals, ...",-1,-1,dessert
253,Mawa Bati,"Milk powder, dry fruits, arrowroot powder, all...",20,45,dessert


In [4]:
#data exploration & cleaning
print(data.course.unique())
print(data.loc[data['course']=='snack'])

['dessert' 'main course' 'starter' 'snack']
                  name                                        ingredients  \
88                Poha  Beaten rice flakes, potato, curry leaves, gree...   
89                Fara  Chana dal, whole wheat flour, arhar dal, white...   
90             Kachori  Moong dal, rava, garam masala, dough, fennel s...   
109          Pani puri      Kala chana, mashed potato, boondi, sev, lemon   
111              Papad       Urad dal, sev, lemon juice, chopped tomatoes   
117             Samosa  Potatoes, green peas, garam masala, ginger, dough   
124               Attu  Whole wheat flour, arhar dal, ginger, kala jee...   
128               Dosa  Chana dal, urad dal, whole urad dal, blend ric...   
129           Idiappam              Rice flour, hot water, grated coconut   
130               Idli  Split urad dal, urad dal, idli rice, thick poh...   
131              Kanji     Carrot, yellow mustard, red chilli, black salt   
144        Masala Dosa  Chana da

In [5]:
#finding data types
print(data.dtypes)

name           object
ingredients    object
prep_time       int64
cook_time       int64
course         object
dtype: object


In [6]:
#removing spaces from string and lowering the case
str_cols = list(data.columns)
str_cols.remove('prep_time')
str_cols.remove('cook_time')
for i in str_cols:
    data[i] = data[i].str.strip()
    data[i] = data[i].str.lower()

In [7]:
import nltk
vocabulary = nltk.FreqDist()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [28]:
#removing missing or correcting garbage value
#data['prep_time'].value_counts()[-1]
#data = data[data.cook_time != -1]
for i in ['name']:
  data[i] = data[i].str.replace(" ", "-")
for i in ['course']:
  data[i] = data[i].str.replace(" ", "-")
for i in ['ingredients']:
  data[i] = data[i].str.replace(" ", "-")
  data[i] = data[i].str.replace(",-", " ")
data['ingredients'] = list(data['ingredients'])

In [29]:
data['ingredients']

0                           maida-flour yogurt oil sugar
1                                  gram-flour ghee sugar
2                carrots milk sugar ghee cashews raisins
3      flour ghee kewra milk clarified-butter sugar a...
4      milk-powder plain-flour baking-powder ghee mil...
                             ...                        
250                glutinous-rice black-sesame-seeds gur
251    coconut-milk egg-yolks clarified-butter all-pu...
252    cottage-cheese dry-dates dried-rose-petals pis...
253    milk-powder dry-fruits arrowroot-powder all-pu...
254    brown-rice fennel-seeds grated-coconut black-p...
Name: ingredients, Length: 255, dtype: object

In [56]:
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
import gensim

[nltk_data] Downloading package punkt to /Users/shruti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [61]:
data['CBOW'] = data['ingredients']+" "+data['name']
data

,name,ingredients,prep_time,cook_time,course,new,CBOW
0,balu-shahi,maida-flour yogurt oil sugar,45,25,dessert,"[maida-flour, yogurt, oil, sugar]",maida-flour yogurt oil sugar balu-shahi
1,boondi,gram-flour ghee sugar,80,30,dessert,"[gram-flour, ghee, sugar]",gram-flour ghee sugar boondi
2,gajar-ka-halwa,carrots milk sugar ghee cashews raisins,15,60,dessert,"[carrots, milk, sugar, ghee, cashews, raisins]",carrots milk sugar ghee cashews raisins gajar-...
3,ghevar,flour ghee kewra milk clarified-butter sugar a...,15,30,dessert,"[flour, ghee, kewra, milk, clarified-butter, s...",flour ghee kewra milk clarified-butter sugar a...
4,gulab-jamun,milk-powder plain-flour baking-powder ghee mil...,15,40,dessert,"[milk-powder, plain-flour, baking-powder, ghee...",milk-powder plain-flour baking-powder ghee mil...
...,...,...,...,...,...,...,...
250,til-pitha,glutinous-rice black-sesame-seeds gur,5,30,dessert,"[glutinous-rice, black-sesame-seeds, gur]",glutinous-rice black-sesame-seeds gur til-pitha
251,bebinca,coconut-milk egg-yolks clarified-butter all-pu...,20,60,dessert,"[coconut-milk, egg-yolks, clarified-butter, al...",coconut-milk egg-yolks clarified-butter all-pu...
252,shufta,cottage-cheese dry-dates dried-rose-petals pis...,-1,-1,dessert,"[cottage-cheese, dry-dates, dried-rose-petals,...",cottage-cheese dry-dates dried-rose-petals pis...
253,mawa-bati,milk-powder dry-fruits arrowroot-powder all-pu...,20,45,dessert,"[milk-powder, dry-fruits, arrowroot-powder, al...",milk-powder dry-fruits arrowroot-powder all-pu...


In [62]:
data1=[]
for i in range(len(data)):
    data1.append(word_tokenize(data['CBOW'][i]))   

In [63]:
data1

[['maida-flour', 'yogurt', 'oil', 'sugar', 'balu-shahi'],
 ['gram-flour', 'ghee', 'sugar', 'boondi'],
 ['carrots', 'milk', 'sugar', 'ghee', 'cashews', 'raisins', 'gajar-ka-halwa'],
 ['flour',
  'ghee',
  'kewra',
  'milk',
  'clarified-butter',
  'sugar',
  'almonds',
  'pistachio',
  'saffron',
  'green-cardamom',
  'ghevar'],
 ['milk-powder',
  'plain-flour',
  'baking-powder',
  'ghee',
  'milk',
  'sugar',
  'water',
  'rose-water',
  'gulab-jamun'],
 ['sugar-syrup', 'lentil-flour', 'imarti'],
 ['maida',
  'corn-flour',
  'baking-soda',
  'vinegar',
  'curd',
  'water',
  'turmeric',
  'saffron',
  'cardamom',
  'jalebi'],
 ['cashews', 'ghee', 'cardamom', 'sugar', 'kaju-katli'],
 ['milk', 'cottage-cheese', 'sugar', 'kalakand'],
 ['milk', 'rice', 'sugar', 'dried-fruits', 'kheer'],
 ['gram-flour', 'ghee', 'sugar', 'laddu'],
 ['yogurt', 'milk', 'nuts', 'sugar', 'lassi'],
 ['refined-flour',
  'besan',
  'ghee',
  'powdered-sugar',
  'yoghurt',
  'green-cardamom',
  'nankhatai'],
 ['fir

In [73]:
def get_window(corpus):
    lengths = [len(doc) for doc in corpus]
    avg_len = float(sum(lengths)) / len(lengths)
    return round(avg_len)

In [96]:
word2Vec=gensim.models.Word2Vec(data1,sg=0, workers=8, window=get_window(data), min_count=1, vector_size=100) 

In [100]:
print(word2Vec.wv.most_similar('peanuts'))

[('keerai-masiyal', 0.3017286956310272), ('pinaca', 0.27092793583869934), ('mung-bean', 0.265238493680954), ('lentils', 0.26371654868125916), ('sev-tameta', 0.2376546412706375), ('chicken-chunks', 0.23450572788715363), ('glutinous-rice', 0.23422379791736603), ('green-bell-pepper', 0.23345784842967987), ('lilva-kachori', 0.2284139096736908), ('white-sesame-seeds', 0.2234800010919571)]


In [101]:
word2Vec.save('model_cbow.bin')

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

In [105]:
class TfidfEmbeddingVectorizer(object):
    def __init__(self, model_cbow):

        self.word2Vec = word2Vec
        self.word_idf_weight = None
        self.vector_size = word2Vec.wv.vector_size

    def fit(self, docs): 
        text_docs = []
        for doc in docs:
            text_docs.append(" ".join(doc))

        tfidf = TfidfVectorizer()
        tfidf.fit(text_docs)  
        # if a word was never seen it is given idf of the max of known idf value
        max_idf = max(tfidf.idf_)  
        self.word_idf_weight = defaultdict(
            lambda: max_idf,
            [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()],
        )
        return self

    def transform(self, docs): 
        doc_word_vector = self.doc_average_list(docs)
        return doc_word_vector

    def doc_average(self, doc):
        mean = []
        for word in doc:
            if word in self.word2Vec.wv.index_to_key:
                mean.append(
                    self.word2Vec.wv.get_vector(word) * self.word_idf_weight[word]
                ) 

        if not mean:  
            return np.zeros(self.vector_size)
        else:
            mean = np.array(mean).mean(axis=0)
            return mean

    def doc_average_list(self, docs):
        return np.vstack([self.doc_average(doc) for doc in docs])

In [112]:
def get_recommendations(N, scores):
    """
    Rank scores and output a pandas data frame containing all the details of the top N recipes.
    :param scores: list of cosine similarities
    """
    # load in recipe dataset
    df_recipes = data
    # order the scores with and filter to get the highest N scores
    top = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:N]
    # create dataframe to load in recommendations
    recommendation = pd.DataFrame(columns=["name", "ingredients", "score"])
    count = 0
    for i in top:
        recommendation.at[count, "name"] = df_recipes["name"][i])
        recommendation.at[count, "ingredients"] = ingredient_parser_final(
            df_recipes["ingredients"][i]
        )
        recommendation.at[count, "score"] = f"{scores[i]}"
        count += 1
    return recommendation

In [125]:
import string
import re
def ingredient_parser(ingredients):
    # measures and common words (already lemmatized)   
    measures = ['teaspoon', 't', 'tsp.', 'tablespoon', 'T', ...]
    words_to_remove = ['fresh', 'oil', 'a', 'red', 'bunch', ...]
    # Turn ingredient list from string into a list 
    if isinstance(ingredients, list):
       ingredients = ingredients
    else:
       ingredients = ast.literal_eval(ingredients)
    # We first get rid of all the punctuation
    translator = str.maketrans('', '', string.punctuation)
    # initialize nltk's lemmatizer    
    lemmatizer = WordNetLemmatizer()
    ingred_list = []
    for i in ingredients:
        i.translate(translator)
        # We split up with hyphens as well as spaces
        items = re.split(' |-', i)
        # Get rid of words containing non alphabet letters
        items = [word for word in items if word.isalpha()]
        # Turn everything to lowercase
        items = [word.lower() for word in items]
        # remove accents
        #items = [unidecode.unidecode(word) for word in items]
        # Lemmatize words so we can compare words to measuring words
        items = [lemmatizer.lemmatize(word) for word in items]
        # get rid of stop words
        #stop_words = set(corpus.stopwords.words('english'))
        #items = [word for word in items if word not in stop_words]
        # Gets rid of measuring words/phrases, e.g. heaped teaspoon
        #items = [word for word in items if word not in measures]
        # Get rid of common easy words
        #items = [word for word in items if word not in words_to_remove]
        if items:
           ingred_list.append(' '.join(items))
    return ingred_list


In [131]:
def get_recs(ingredients, N=5, mean=False):
    """
    Get the top N recipe recomendations.
    :param ingredients: comma seperated string listing ingredients
    :param N: number of recommendations
    :param mean: False if using tfidf weighted embeddings, True if using simple mean
    """
    # load in word2vec model
    model = Word2Vec.load("model_cbow.bin")
    # normalize embeddings
    model.init_sims(replace=True)
    if model:
        print("Successfully loaded model")
    # load in data
    data = data1
    # parse ingredients
    #data["parsed"] = data.ingredients.apply(ingredient_parser)
    # create corpus
    #corpus = get_and_sort_corpus(data)

    if mean:
        # get average embdeddings for each document
        mean_vec_tr = MeanEmbeddingVectorizer(model)
        doc_vec = mean_vec_tr.transform(corpus)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(corpus)
    else:
        # use TF-IDF as weights for each word embedding
        tfidf_vec_tr = TfidfEmbeddingVectorizer(model)
        tfidf_vec_tr.fit(data)
        doc_vec = tfidf_vec_tr.transform(data)
        doc_vec = [doc.reshape(1, -1) for doc in doc_vec]
        assert len(doc_vec) == len(data)

    # create embeddings for input text
    input = ingredients
    # create tokens with elements
    input = input.split(",")
    # parse ingredient list
    input = ingredient_parser(input)
    # get embeddings for ingredient doc
    if mean:
        input_embedding = mean_vec_tr.transform([input])[0].reshape(1, -1)
    else:
        input_embedding = tfidf_vec_tr.transform([input])[0].reshape(1, -1)

    # get cosine similarity between input embedding and all the document embeddings
    cos_sim = map(lambda x: cosine_similarity(input_embedding, x)[0][0], doc_vec)
    scores = list(cos_sim)
    # Filter top N recommendations
    recommendations = get_recommendations(N, scores)
    return recommendations
  
if __name__ == "__main__":
    # test
    input = "fenugreek,salt,yogurt,besan"
    rec = get_recs(input)
    print(rec)

Successfully loaded model
           name                                        ingredients  \
0   gatta-curry  yogurt besan sauce garam-masala-powder gram-flour   
1  kadhi-pakoda  besan garam-masala-powder gram-flour ginger cu...   
2  bandar-laddu  besan jaggery cardamom-powder ghee cashews-and...   
3         lassi                             yogurt milk nuts sugar   
4    aloo-methi          potato fenugreek-leaves chillies salt oil   

                 score  
0   0.5150182843208313  
1  0.45304176211357117  
2  0.40088143944740295  
3   0.3924981355667114  
4    0.338057279586792  


/var/folders/v8/mdvtnyk13yv9d_65fyxl0yt00000gn/T/ipykernel_78378/1559240830.py:11: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  model.init_sims(replace=True)


In [12]:
from flask import Flask, render_template, url_for, request, redirect
app = Flask(__name__)


@app.route('/', methods=['POST','GET'])
def index():
    if request.method == 'POST':
        ingredients = request.form.get('fname')
        res = easyA(ingredients)
        return render_template("dishrec.html", res=res)

    else: 
        return render_template("dishrec.html")

if __name__ == '__main__':
    app.run(debug = True)
    


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/shruti/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/shruti/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 18, in <module>
    from IPython.core.application import (
  File "/Users/shruti/opt/anaconda3/lib/python3.9/site-packages/IPython/__init__.py", line 56, in <module>
    from .terminal.embed import embed
  File "/Users/shruti/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/embed.py", line 16, in <module>
    from IPython.terminal.interactiveshell import TerminalInteractiveShell
  File "/Users/shruti/opt/anaconda3/lib/python3.9/site-packages/IPython/terminal/interactiveshell.py", line 35, in <module>
    from .debugger import TerminalPdb, Pdb
  File "/Users/shruti/opt/anaconda3/lib/python3.9/site-packag

SystemExit: 1

/Users/shruti/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
flask run